### Delta calculation algo (for now start from (3))
1. On first iteration of trajectory - calculate p, q vectors using identify
2. For each trajectory try to use the p, q vectors calculated before, if converges good using these - good! else run identify again.
3. Given the correct p, q vectors and limit (i.e. theoretical limit - e.g. pi), does our limit converges to the theoretical one. how do we do that? try one step before and after (999, 1000, 1001) and compare limits. difference should be very very small (e.g. 1/2^1000 - should be tested) - this tests general convegence. To test convergence to the constant test the same with the constant.

(read about walk for a matrix)

In [1]:
from ramanujantools.cmf.pfq import pFq
from ramanujantools import Limit, Matrix
import sympy as sp
from LIReC.db.access import db

In [2]:
x0, x1, y0, n = sp.symbols('x0 x1 y0 n')


# This is pi 2F1 CMF
pi = pFq(2, 1, sp.Rational(1, 2))


def calc_delta(cmf, traj, strt):
    # Do walk
    traj_m = pi.trajectory_matrix(traj, strt)
    walked = traj_m.walk({n: 1}, 1000, {n: 0})
    walked = walked.inv().T
    t1_col = (walked / walked[0, 0]).col(0)
    values = t1_col[1:]

    # Identify and calculate delta
    res = db.identify([sp.pi.evalf(300)] + values)
    """
    Use a dictionary to check each time if
    """

    if res:
        # convert result to sympy expression
        res = res[0]
        print(res)
        res.include_isolated = 0
        res = '('.join(str(res).split('(')[:-1])
        simpified = sp.sympify(res)

        # estimate pi with the rational
        estimated = simpified.subs({sym: val for sym, val in zip(sp.symbols(f'c:{len(values) + 1}')[1:], values)})

        # compute delta
        error_v = sp.Abs(estimated - sp.pi.evalf(30000))
        denom = sp.denom(estimated)

        if denom == 1:
          raise ZeroDivisionError('Denominator 1 caused zero division in delta calculation')
        if denom < 1e6:
          raise Exception("Probably still rational")

        delta = -1 - sp.log(error_v) / sp.log(denom)

        return delta.evalf(10), estimated.evalf(15)
    return None, None

In [ ]:
start1 = {x0: sp.Rational(9,2), x1: sp.Rational(11, 2), y0: sp.Rational(15, 2)}
start2 = {x0: sp.Rational(-9, 2), x1: sp.Rational(-3, 2), y0: sp.Rational(5, 2)}
trajectory = {x0: -4, x1: -3, y0: 2}

print(calc_delta(pi, trajectory, start1))
print(calc_delta(pi, trajectory, start2))



In [ ]:
def does_converge(t_limit, t_mat: Matrix, p, q) -> bool:
    limits = [l.as_float() for l in t_mat.limit({n: 1}, [999, 1000, 1001], {n: 0}, initial_values=Matrix([p, q]))]
    diff1 = abs(limits[0] - limits[1])
    diff2 = abs(limits[2] - limits[1])




In [3]:
pi.matrices

{x0: Matrix([
 [   1,                          x1],
 [1/x0, 1 + (x0 + x1 - 2*y0 + 2)/x0]]),
 x1: Matrix([
 [   1,                          x0],
 [1/x1, 1 + (x0 + x1 - 2*y0 + 2)/x1]]),
 y0: Matrix([
 [-y0*(x0 + x1 - y0)/((x0 - y0)*(x1 - y0)), x0*x1*y0/((x0 - y0)*(x1 - y0))],
 [                y0/((x0 - y0)*(x1 - y0)),   -y0**2/((x0 - y0)*(x1 - y0))]])}

In [10]:
sp.solve(pi.matrices[x1].det())

[{x1: y0 - 1}]